### loading dataset

https://www.kaggle.com/competitions/titanic/data

In [ ]:
# Import necessary Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)


from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read Data
data = pd.read_csv(r'../../../../../1. titanic_dataset\1. train.csv')
data.head()


## preparing cols to be used

In [ ]:
data

### checking and fixing missing values in all cols

In [ ]:
(data.isna().sum()/data.shape[0])*100

In [ ]:
data['Age'].fillna(data['Age'].mean, inplace=True)

In [ ]:
data.drop(columns=['Cabin'], axis=1, inplace=True)

In [ ]:
(data.isna().sum()/data.shape[0])*100

In [ ]:
data.sample(3)

### checking corr among cols for feature selection

In [ ]:
data[['Fare', 'Survived']].corr()

### vectorizing cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
encoded_sex = ohe.fit_transform(data[['Sex']])
df = pd.DataFrame(data=encoded_sex.toarray(), columns=['male', 'female'])
pd.concat([data, df], axis=1)

## modeling

### building parts of algo

In [ ]:
# mean
def get_mean(arr):
    return np.sum(arr)/len(arr)

# variance
def get_variance(arr, mean):
    return np.sum((arr-mean)**2)

# covariance
def get_covariance(arr_x, mean_x, arr_y, mean_y):
    final_arr = (arr_x - mean_x)*(arr_y - mean_y)
    return np.sum(final_arr)

In [ ]:
# find coeff
def get_coefficients(x, y):
    x_mean = get_mean(x)
    y_mean = get_mean(y)
    m = get_covariance(x, x_mean, y, y_mean)/get_variance(x, x_mean)
    c = y_mean - x_mean*m
    return m, c

# Regression Function
def linear_regression(x_train, y_train, x_test, y_test):
    prediction = []
    m, c = get_coefficients(x_train, y_train)
    for x in x_test:
        y = m*x + c
        prediction.append(y)

    r2 = r2_score(prediction, y_test)
    mse = mean_squared_error(prediction, y_test)
    print("The R2 score of the model is: ", r2)
    print("The MSE score of the model is: ", mse)
    return prediction




### training model

In [ ]:
x = data['Age'].values
y = data['Survived'].values

In [ ]:
# There are 100 sample out of which 80 are for training and 20 are for testing
lr = linear_regression(x[:80], y[:80], x[80:], y[80:])

### visualizing linear regression algo learning process

In [ ]:
# Visualize
def plot_reg_line(x, y):
    prediction = []
    m, c = get_coefficients(x, y)
    for x0 in range(1,100):
        yhat = m*x0 + c
        prediction.append(yhat)

    fig = plt.figure(figsize=(20,7))
    plt.subplot(1,2,1)
    sns.scatterplot(x=x, y=y)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Scatter Plot between X and Y')

    plt.subplot(1,2,2)
    sns.scatterplot(x=x, y=y, c='blue')
    sns.lineplot(x = [i for i in range(1, 100)], y = prediction, c='red')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Regression Plot')
    plt.show()

plot_reg_line(x, y)